In [7]:
import pandas as pd
import numpy as np

tags = pd.read_csv('tags.csv')
rating = pd.read_csv('ratings.csv')
to_read = pd.read_csv('to_read.csv')
book = pd.read_csv('book_no_null.csv')
book_tags = pd.read_csv('book_tags.csv')

## Data cleaning

In [2]:
book.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


### Missing values 

### Checking the missing values

In [3]:
book.isnull().sum()

book_id                         0
goodreads_book_id               0
best_book_id                    0
work_id                         0
books_count                     0
isbn                          700
isbn13                        585
authors                         0
original_publication_year      21
original_title                585
title                           0
language_code                1084
average_rating                  0
ratings_count                   0
work_ratings_count              0
work_text_reviews_count         0
ratings_1                       0
ratings_2                       0
ratings_3                       0
ratings_4                       0
ratings_5                       0
image_url                       0
small_image_url                 0
dtype: int64

In [5]:
import sys
from isbntools.app import *
import pyisbn
import time


### Imputation and correct isbn13 data

In [5]:
def isbn_creator(strat_indx, end_indx):
    exception_isbn = []
    for i in book[(book.book_id < end_indx) & (book.book_id >= strat_indx)].index:
        try:
            isbn13 = isbn_from_words(str(book.loc[i].title + ' ' + book.loc[i].authors))
            book.loc[i,'isbn13'] = isbn13
            time.sleep(0.2)
        except:
            exception_isbn.append(i)
            print(i, "Next entry")
    return exception_isbn

In [6]:
def second_isbn_creator(unavailables):
    exception_isbn = []
    for i in unavailables:
        try:
            isbn13 = isbn_from_words(str(book.loc[i].title + ' ' + book.loc[i].authors))
            book.loc[i,'isbn13'] = isbn13
            time.sleep(0.2)
        except:
            exception_isbn.append(i)
            print(i, "Next entry")
    return exception_isbn

In [16]:
unavailables = []
for i in range(0, 10000, 500):
    exception_isbn = isbn_creator(i, i+501)
    for exc in exception_isbn:
        unavailables.append(exc)

print (len(unavailables), ' book is not available for getting isbn13')
print ('____________________________________________________________')
print (unavailables)

67  book is not available for getting isbn13
____________________________________________________________
[193, 395, 1178, 2024, 2811, 3145, 3296, 3505, 3527, 3655, 3753, 3864, 3907, 4008, 4044, 4070, 4164, 4204, 4325, 4393, 4412, 4466, 4514, 4516, 4733, 4764, 4809, 4874, 5171, 5232, 5265, 5325, 5390, 5947, 6063, 6096, 6134, 6199, 6233, 6254, 6255, 6256, 6257, 6259, 6261, 6263, 6366, 6528, 7393, 7503, 7718, 8083, 8342, 8379, 8408, 8919, 9084, 9142, 9327, 9371, 9386, 9521, 9666, 9690, 9766, 9771, 9961]


In [28]:
exception_isbn = second_isbn_creator(unavailables)

print (len(exception_isbn), ' book is not available for getting isbn13')
print ('____________________________________________________________')
print (exception_isbn)

12  book is not available for getting isbn13
____________________________________________________________
[193, 2811, 4044, 4412, 4733, 5171, 6096, 6134, 6233, 7503, 9690, 9766]


In [84]:
for i in range(0, 20):
    exception_isbn = second_isbn_creator(exception_isbn)
    
print (len(exception_isbn), ' book is not available for getting isbn13')
print ('____________________________________________________________')
print (exception_isbn)

0  book is not available for getting isbn13
____________________________________________________________
[]


In [161]:
book.to_csv('book_no_null.csv', index=False)

### Check validity and being null in isbn13 column

In [7]:
def validate_isbn(strat_indx, end_indx):
    IsNull = []
    IsInvalid = []
    for i in book[(book.book_id < end_indx) & (book.book_id >=strat_indx)].index:
        try:
            if pyisbn.validate(str(int(book.loc[i].isbn13))) != True:
                IsInvalid.append(i)
        except:
            IsNull.append(i)
            
    return (IsNull, IsInvalid)

In [127]:
book['isbn13'].replace({'': np.nan}, inplace = True)

In [102]:
IsNull, IsInvalid = validate_isbn(0, 10000)
print (len(IsInvalid), ' isbn is not valid')
print ('____________________________________________________________')
print(IsInvalid)

0  isbn is not valid
____________________________________________________________
[]


In [103]:
print (len(IsNull), ' isbn is null')
print ('____________________________________________________________')
print(IsNull)

84  isbn is null
____________________________________________________________
[726, 1053, 1113, 1179, 1196, 1225, 1772, 1832, 1894, 2164, 2442, 2936, 3198, 3266, 3371, 3394, 3632, 3824, 4005, 4065, 4105, 4160, 4204, 4414, 4441, 4606, 4637, 4675, 4687, 4821, 4891, 4934, 5052, 5209, 5359, 5376, 5646, 5678, 5715, 5868, 6194, 6260, 6451, 6600, 6742, 7084, 7166, 7224, 7471, 7568, 7619, 7638, 7674, 7782, 7934, 8039, 8078, 8108, 8111, 8143, 8246, 8335, 8431, 8482, 8486, 8522, 8758, 8868, 8881, 8912, 9043, 9066, 9121, 9140, 9178, 9220, 9320, 9384, 9557, 9609, 9619, 9677, 9689, 9857]


In [107]:
exception_isbn = second_isbn_creator(IsNull)

print (len(exception_isbn), ' book is not available for getting isbn13')
print ('____________________________________________________________')
print (exception_isbn)

1  book is not available for getting isbn13
____________________________________________________________
[4821]


In [116]:
book['isbn13'].replace({'': np.nan}, inplace = True)

In [129]:
IsNull, IsInvalid = validate_isbn(0, 10000)
print (len(IsInvalid), ' isbn is not valid')
print ('____________________________________________________________')
print(IsInvalid)

0  isbn is not valid
____________________________________________________________
[]


In [130]:
print (len(IsNull), ' isbn is null')
print ('____________________________________________________________')
print(IsNull)

76  isbn is null
____________________________________________________________
[726, 1053, 1113, 1179, 1225, 1832, 1894, 2164, 2442, 2936, 3198, 3371, 3394, 3824, 4005, 4065, 4105, 4160, 4204, 4414, 4441, 4606, 4637, 4675, 4687, 4821, 4891, 4934, 5052, 5209, 5359, 5376, 5646, 5678, 5715, 5868, 6194, 6260, 6451, 6600, 7084, 7166, 7224, 7471, 7568, 7619, 7638, 7674, 7782, 7934, 8039, 8078, 8108, 8111, 8143, 8246, 8335, 8431, 8482, 8486, 8522, 8758, 8868, 8881, 8912, 9043, 9066, 9121, 9140, 9178, 9320, 9384, 9609, 9619, 9677, 9857]


In [ ]:
book.to_csv('book_no_null.csv', index=False)

### Fill null values in original_publication_year

In [145]:
null_publication_year = book[book['original_publication_year'].isna()].index
print(null_publication_year)

Int64Index([ 219,  975, 3505, 4228, 4247, 4409, 4707, 4770, 4877, 5609, 5871,
            6428, 7190, 7215, 7416, 7645, 8476, 9196, 9510, 9533, 9928],
           dtype='int64')


In [185]:
exception_original_publication_year = []
for i in null_publication_year:
    try:
        json_response = meta(str(int(book.loc[i,'isbn13'])))
        book.loc[i,'original_publication_year'] = json_response['Year']
    except:
        exception_original_publication_year.append(i)

In [186]:
exception_original_publication_year

[975, 5871, 6428, 7190]

### Replace missing data with form ' ' by np.nan

In [187]:
book['original_publication_year'].replace({'': np.nan}, inplace = True)

In [188]:
book[book['original_publication_year'].isna()].index

Int64Index([975, 5871, 6428, 7190, 9928], dtype='int64')

### Imputation of remaining null values in original_publication_year

In [4]:
book.loc[975,'original_publication_year'] = 1998

book.loc[5871,'original_publication_year'] = 2010

book.loc[6428,'original_publication_year'] = 2006

book.loc[7190,'original_publication_year'] = 2010

book.loc[9928,'original_publication_year'] = 1998

In [6]:
book.to_csv('book_no_null.csv', index=False)

### See null languages

In [8]:
null_language_code = book[book['language_code'].isna()].index
print(null_language_code)

Int64Index([  44,  131,  203,  236,  287,  320,  408,  412,  464,  486,
            ...
            9951, 9953, 9959, 9967, 9969, 9972, 9973, 9976, 9985, 9999],
           dtype='int64', length=1084)


In [9]:
book['language_code'].unique()

array(['eng', 'en-US', 'en-CA', nan, 'spa', 'en-GB', 'fre', 'nl', 'ara',
       'por', 'ger', 'nor', 'jpn', 'en', 'vie', 'ind', 'pol', 'tur',
       'dan', 'fil', 'ita', 'per', 'swe', 'rum', 'mul', 'rus'],
      dtype=object)

### Imputation of Language code column

In [10]:
exception_language_code = []
lan_code = []
for i in null_language_code:
    try:
        lan_response = info(str(int(book.loc[i,'isbn13'])))
        book.loc[i,'language_code'] = lan_response
        lan_code.append(lan_response)
    except:
        exception_language_code.append(i)

In [11]:
exception_language_code

[1053, 2164, 5376, 9677]

In [12]:
book['language_code'].replace({'': np.nan}, inplace = True)

In [13]:
def unique(list1):
    
    unique_list = []
    for x in list1:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

In [14]:
unique(lan_code)

['English language',
 "China, People's Republic",
 'United States',
 'Turkey',
 'Sweden',
 'German language',
 'India',
 'Finland',
 'French language',
 'Japan',
 'Argentina']

In [15]:
book['language_code'].replace({  '': np.nan, 
                                 'English language':'eng',
                                 "China, People's Republic":'chi',
                                 'United States':'en-US',
                                 'Turkey':'tur',
                                 'Sweden':'swe',
                                 'German language':'ger',
                                 'India':'hin',
                                 'Finland':'fin',
                                 'French language':'fre',
                                 'Japan':'jpn',
                                 'Argentina': 'es-ar'
                                }, inplace = True)

In [17]:
book[book['language_code'].isna()].index

Int64Index([1053, 2164, 5376, 9677], dtype='int64')

### Imputation of remaining null values in language_code

In [7]:
book.loc[1053,'language_code'] = 'eng'

book.loc[2164,'language_code'] = 'eng'

book.loc[5376,'language_code'] = 'fre'

book.loc[9677,'language_code'] = 'eng'

In [8]:
book.to_csv('book_no_null.csv', index=False)

### Change eng,en-GB,en-CA,en-US to en

In [10]:
book.language_code = book.language_code.replace(['eng','en-CA','en-GB','en-US'],'en')

In [11]:
book['language_code'].unique()

array(['en', 'spa', 'fre', 'nl', 'ara', 'por', 'ger', 'nor', 'chi', 'jpn',
       'vie', 'ind', 'pol', 'tur', 'swe', 'dan', 'fil', 'ita', 'per',
       'hin', 'fin', 'es-ar', 'rum', 'mul', 'rus'], dtype=object)

In [12]:
book.to_csv('book_no_null_language_merging.csv', index=False)

In [13]:
book.isna().sum()

book_id                        0
goodreads_book_id              0
best_book_id                   0
work_id                        0
books_count                    0
isbn                         700
isbn13                        76
authors                        0
original_publication_year      0
original_title               585
title                          0
language_code                  0
average_rating                 0
ratings_count                  0
work_ratings_count             0
work_text_reviews_count        0
ratings_1                      0
ratings_2                      0
ratings_3                      0
ratings_4                      0
ratings_5                      0
image_url                      0
small_image_url                0
dtype: int64